BASES NT

Concatenação das bases de dados

In [224]:
import pandas as pd
import pandas

# Carrega os DataFrames dos arquivos Excel de 2022 e 2023
df22 = pd.read_excel('c:\\Users\\caiobarreto\\Downloads\\Relatório_NT_2022.xlsx')
df23 = pd.read_excel('c:\\Users\\caiobarreto\\Downloads\\Relatório_NT_2023.xlsx')
# Concatena os DataFrames
df = pd.concat([df22, df23])
# Define o caminho do arquivo de saída para o DataFrame concatenado
saida = 'c:\\Users\\caiobarreto\\Downloads\\Relatório_NT_Concatenado.xlsx'
# Salva o DataFrame concatenado em um novo arquivo Excel
df.to_excel(saida, index=False)

Retirar acentos e ç

In [225]:
from unidecode import unidecode

# Remover acentos e caracteres especiais de todas as colunas
df = df.apply(lambda x: x.map(lambda y: unidecode(str(y)) if isinstance(y, str) else y))

 Ele utiliza a função apply para aplicar uma função lambda a cada coluna do DataFrame, e dentro dessa função lambda, utiliza outra função lambda para percorrer cada valor da coluna e remover a acentuação, caso o valor seja uma string.

Padronizar formato de data e hora

In [226]:
# Padronizar o formato de data e hora
formatar = ['Data Prevista de Chegada', 'Data Prevista de Saída', 
'H.O.C.', 'Amarração','Início de Bombeio', 'Fim de Bombeio', 'Desamarração', 'H.O.S.', 'Início Primeira Amarração','Fim Primeira Amarração','Início Primeira Conexão','Fim Primeira Conexão',
'Início Primeiro Bombeio','Fim Último Bombeio','Início Última Desconexão','Fim Última Desconexão','Início Última Desamarração','Fim Última Desamarração']

for coluna in formatar:
    df[coluna] = pandas.to_datetime(df[coluna], dayfirst=True)

Remover colunas

In [227]:
# Removendo as colunas, inplace é pra não precisar reatribuir e axis 1 serve pra dizer que se trata de uma coluna
df.drop(['Classe do Navio', 'Rebocador', 'Hora Top', 'Alívio Crítico', 'Apuração Alívio Crítico', 'Volume Acumulado Perdas', 'Lifter'], inplace=True, axis=1)

Tirar duplicatas

In [228]:
# Remove as duplicatas
df=df.drop_duplicates()

Adicionando coluna ID e adicionando valor ao indice

In [229]:
# Redefinir o índice do DataFrame
df.reset_index(drop=True, inplace=True)
# Adicionando a coluna
df.insert(0,'ID',range(len(df)))

Criando coluna UEP e trocando Petrobras por P-

In [230]:
# Adicionando a coluna UEP e pegando o valor da coluna Instalação Naval
df.insert(2, 'UEP', df['Instalação Naval'].str.replace('PETROBRAS', 'P-', regex=True))

Retirar os espaços em branco antes e no fim das colunas UEP e Navio

In [231]:
# Remover espaços em branco antes e depois dos nomes nas colunas UEP e Navio
df['UEP'] = df['UEP'].str.strip()
df['Navio'] = df['Navio'].str.strip()

Transformar H.O.C em data prevista de chegada quando a diferença for >= 48 horas

In [232]:
# Calculando a diferença em horas entre as datas previstas de chegada e o H.O.C
diferenca_horas = (df['Data Prevista de Chegada'] - df['H.O.C.']).dt.total_seconds() / 3600

# Substituindo os valores de "H.O.C" onde a diferença for maior do que 48 horas (2 dias)
df.loc[diferenca_horas >= 48, 'H.O.C.'] = df['Data Prevista de Chegada']

Tirar duplicatas dos casos que possuem mesmo HOC, Instalação Naval e Navio

In [233]:
# Remover duplicatas baseadas em "H.O.C", "Instalação Naval" e "Navio"
df.drop_duplicates(subset=['H.O.C.', 'Instalação Naval', 'Navio'], inplace=True)

ALTERANDO O FORMATO DA BASE

In [234]:
# Selecionar apenas as colunas necessárias para alteração no formato do dataframe
nt_colunas = ['ID', 'H.O.C.', 'Início Primeira Amarração', 'Fim Primeira Amarração', 'Início Primeira Conexão', 'Fim Primeira Conexão', 'Início Primeiro Bombeio','Fim Último Bombeio', 'Início Última Desconexão', 'Fim Última Desconexão','Início Última Desamarração', 'Fim Última Desamarração', 'H.O.S.']

# Selecionar e definir as colunas que serão alteradas
colunas_alteradas = ['H.O.C.', 'Início Primeira Amarração', 'Fim Primeira Amarração','Início Primeira Conexão', 'Fim Primeira Conexão', 'Início Primeiro Bombeio','Fim Último Bombeio', 'Início Última Desconexão', 'Fim Última Desconexão','Início Última Desconexão', 'Fim Última Desconexão', 'Início Última Desamarração','Fim Última Desamarração', 'H.O.S.']
# Usar o método melt para transformar as colunas em registros
df_transformado = pd.melt(df[nt_colunas], id_vars=['ID'], value_vars=colunas_alteradas, var_name='Ocorrência', value_name='Data')
# Selecionar apenas as colunas necessárias para a junção
colunas_restauradas = ['ID', 'Unidade Operativa', 'UEP', 'Navio', 'Volume Programado','Volume Retirado', 'Viagem', 'Destino', 'Data Prevista de Chegada','Data Prevista de Saída']
# Realizar a junção dos DataFrames usando a coluna 'ID' como chave
df_final = pd.merge(df[colunas_restauradas], df_transformado, on='ID', how='right')


pd.melt() é usado para transformar as colunas selecionadas em registros. O argumento id_vars=['ID'] indica quais colunas manter como identificadores, enquanto value_vars=colunas_alteradas especifica quais colunas devem ser transformadas em registros. As colunas "Ocorrência" e "Data" são os nomes das novas colunas geradas durante a transformação.

Ordenar a base por ID e Data

In [235]:
# Ordenar o DataFrame final por 'ID' e 'Data'
df_final = df_final.sort_values(by=['ID', 'Data'])

Adicionar os respectivos valores às colunas ‘Início’ e ‘Término’ de cada ocorrência

In [236]:
# Adicionar as colunas 'Início' e 'Término' com base nas datas da ocorrência e da próxima ocorrência para o mesmo ID
df_final['Início'] = df_final['Data']
# Obter o valor da próxima linha na coluna "Início"
df_final['Término'] = df_final['Início'].shift(-1)

Excluir as colunas com “Ocorrência” = ”Fim”

In [237]:
# Filtrar as linhas em que a "Ocorrência" contém a palavra "Fim" e obter seus índices
indices_para_remover = df_final[df_final['Ocorrência'].str.contains('Fim')].index
# Remover as linhas correspondentes aos índices obtidos
df_final.drop(indices_para_remover, inplace=True)

Renomear as colunas de Início 

In [238]:
# Retirar a palavra Início da ocorrência
df_final['Ocorrência'] = df_final['Ocorrência'].str.replace('Início ', '')

Remover a coluna data

In [239]:
# Removendo as colunas, inplace é pra não precisar reatribuir e axis 1 serve pra dizer que se trata de uma coluna
df_final.drop(['Data'],inplace=True, axis=1)

BASE INTERRUPÇÕES

Importar Base Interrupções

In [240]:
dfInt = pd.read_excel("c:\\Users\\caiobarreto\\Downloads\\eventos_operacoes_interrupcoes2.xlsx")

Filtrar somente as que tiveram interrupções

In [241]:
# Filtrar apenas as linhas em que 'Duração Interrupção' é maior que zero e reatribuir ao DataFrame dfInt
dfInt = dfInt[dfInt['Duração Interrupção'] > 0]

Retirando Acento e Ç

In [242]:
dfInt = dfInt.apply(lambda x: x.map(lambda y: unidecode(str(y)) if isinstance(y, str) else y))

Criando coluna UEP

In [243]:
# Adicionando a coluna UEP 
dfInt.insert(2, 'UEP', dfInt['INNA_SG_INSTALACAO_NAVAL'].str.replace('PETROBRAS', 'P-', regex=True))

Alterando nomes das categorias da coluna UEP

In [244]:
# Faz um dicionário com os nomes a serem mudados na UEP
trocaNomes = {
    'FPBAR': 'FPSO Almirante Barroso',
    'FPCGZ': 'FPSO Campos dos Goytacases MV29',
    'CAPX': 'FPSO Capixaba',
    'FPCRC': 'FPSO CARIOCA MV30',
    'FPNIT': 'FPSO Cid. Niteroi MV18',
    'CDAN': 'FPSO Cidade de Anchieta',
    'FPCAR': 'FPSO Cidade de Angra Reis MV22',
    'FPCIB': 'FPSO Cidade de Ilhabela',
    'FPCMB': 'FPSO Cidade de Mangaratiba MV24',
    'FPCIG': 'FPSO Cidade de Itaguai MV26',
    'FPCMC': 'FPSO Cidade de Marica',
    'FPCPY': 'FPSO Cidade de Paraty',
    'FPCST': 'FPSO Cidade de Santos MV20',
    'FPCSQ': 'FPSO Cidade de Saquarema',
    'FPCSP': 'FPSO Cidade Sao Paulo MV23',
    'FPGNB': 'FPSO Guanabara - MV31',
    'FPPLB': 'FPSO Pioneiro de Libra',
    'FSME': 'FSO Cidade de Macae - MV15'
}

# Faz a troca dos nomes da UEP utilizando o replace
dfInt['UEP'] = dfInt['UEP'].replace(trocaNomes)

Renomear as colunas para ficar com a mesma nomenclatura que a base NT

In [245]:
# Definindo o dicionário para renomear as colunas
nomeColunas = {
    'OPOF_CD_VIAGEM': 'Viagem',
    'OPOF_NM_NAVIO': 'Navio',
    'OPOF_NM_DESTINO': 'Destino',
    'EOOF_IN_TIPO': 'TIPO_INTERRUPCAO',
    'Duração Interrupção': 'DURACAO_INTERRUPCAO',
    'EOOF_DT_REALIZADA': 'INICIO_INTERRUPCAO',
    'EOOF_DT_FIM': 'FIM_INTERRUPCAO',
    'OPOF_DT_ENTRADA_PREVISTA': 'Data Prevista de Chegada',
    'OPOF_DT_SAIDA_PREVISTA': 'Data Prevista de Saída'
}

# Renomeando as colunas conforme o dicionário 
dfInt.rename(columns=nomeColunas, inplace=True)

Retirar o espaço do inicio e final das colunas Navio e UEP

In [246]:
# Remover espaços em branco antes e depois dos nomes nas colunas UEP e Navio
dfInt['UEP'] = dfInt['UEP'].str.strip()
dfInt['Navio'] = dfInt['Navio'].str.strip()

Selecionar apenas as colunas que interessam

In [247]:
# Selecionar apenas as colunas necessárias do DataFrame dfInt
colunasSel = dfInt[['UEP', 'Viagem', 'Navio', 'Destino', 'TIPO_INTERRUPCAO', 'DURACAO_INTERRUPCAO', 'INICIO_INTERRUPCAO', 'FIM_INTERRUPCAO', 'Data Prevista de Chegada', 'Data Prevista de Saída']]

JUNTAR BASE DE INTERRUPÇÕES COM A DE NAVIO TANQUE

In [248]:
# Junção das bases de interrupções e NT
dfUni = pd.merge(df_final, colunasSel, on=['UEP', 'Viagem', 'Navio', 'Destino', 'Data Prevista de Chegada', 'Data Prevista de Saída'], how='left')
dfUni

,ID,Unidade Operativa,UEP,Navio,Volume Programado,Volume Retirado,Viagem,Destino,Data Prevista de Chegada,Data Prevista de Saída,Ocorrência,Início,Término,TIPO_INTERRUPCAO,DURACAO_INTERRUPCAO,INICIO_INTERRUPCAO,FIM_INTERRUPCAO
0,0,UN-BS,P- 67,FORTALEZA KNUTSEN,80000,81382.81,423 TA,T.B.N.,2022-04-27 12:00:00,2022-04-28 09:30:00,H.O.C.,2022-04-27 03:00:00,2022-04-27 09:04:00,NaN,NaN,NaT,NaT
1,0,UN-BS,P- 67,FORTALEZA KNUTSEN,80000,81382.81,423 TA,T.B.N.,2022-04-27 12:00:00,2022-04-28 09:30:00,Primeira Amarração,2022-04-27 09:04:00,2022-04-27 09:35:00,NaN,NaN,NaT,NaT
2,0,UN-BS,P- 67,FORTALEZA KNUTSEN,80000,81382.81,423 TA,T.B.N.,2022-04-27 12:00:00,2022-04-28 09:30:00,Primeira Conexão,2022-04-27 09:40:00,2022-04-27 10:11:00,NaN,NaN,NaT,NaT
3,0,UN-BS,P- 67,FORTALEZA KNUTSEN,80000,81382.81,423 TA,T.B.N.,2022-04-27 12:00:00,2022-04-28 09:30:00,Primeiro Bombeio,2022-04-27 12:00:00,2022-04-28 07:00:00,NaN,NaN,NaT,NaT
4,0,UN-BS,P- 67,FORTALEZA KNUTSEN,80000,81382.81,423 TA,T.B.N.,2022-04-27 12:00:00,2022-04-28 09:30:00,Última Desconexão,2022-04-28 08:30:00,2022-04-28 08:55:00,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22661,3225,UN-BUZ,P- 76,ELKA LEBLON,80000,81372.15,402 TA,T.B.N.,2023-07-31 13:00:00,2023-08-01 18:00:00,Primeira Conexão,2023-07-31 16:30:00,2023-07-31 17:48:00,NaN,NaN,NaT,NaT
22662,3225,UN-BUZ,P- 76,ELKA LEBLON,80000,81372.15,402 TA,T.B.N.,2023-07-31 13:00:00,2023-08-01 18:00:00,Primeiro Bombeio,2023-07-31 19:06:00,2023-08-01 12:42:00,NaN,NaN,NaT,NaT
22663,3225,UN-BUZ,P- 76,ELKA LEBLON,80000,81372.15,402 TA,T.B.N.,2023-07-31 13:00:00,2023-08-01 18:00:00,Última Desconexão,2023-08-01 14:48:00,2023-08-01 15:36:00,NaN,NaN,NaT,NaT
22664,3225,UN-BUZ,P- 76,ELKA LEBLON,80000,81372.15,402 TA,T.B.N.,2023-07-31 13:00:00,2023-08-01 18:00:00,Última Desamarração,2023-08-01 15:37:00,2023-08-01 16:12:00,NaN,NaN,NaT,NaT


Restringir as características de interrupção apenas aos bombeios

In [249]:
# Substituir valor da coluna 'TIPO_INTERRUPCAO' por 'Interrupção Bombeio' onde 'Ocorrência' contém 'Bombeio' e 'TIPO_INTERRUPCAO' = 'B005', caso contrário, substituir por ''
condicao_1 = (dfUni['Ocorrência'].str.contains('Bombeio')) & (dfUni['TIPO_INTERRUPCAO'] == 'B005')
dfUni.loc[condicao_1, 'TIPO_INTERRUPCAO'] = 'Interrupção Bombeio'
dfUni.loc[~condicao_1, 'TIPO_INTERRUPCAO'] = ''

# Manter valor da coluna 'DURACAO_INTERRUPCAO' onde 'Ocorrência' contém 'Bombeio' e 'DURACAO_INTERRUPCAO' > 0
condicao_2 = (dfUni['Ocorrência'].str.contains('Bombeio')) & (dfUni['DURACAO_INTERRUPCAO'] > 0)
dfUni.loc[condicao_2, 'DURACAO_INTERRUPCAO'] = dfUni.loc[condicao_2, 'DURACAO_INTERRUPCAO']
dfUni.loc[~condicao_2, 'TIPO_INTERRUPCAO'] = ''

# Manter valor da coluna 'INICIO_INTERRUPCAO' onde 'Ocorrência' contém 'Bombeio' e 'TIPO_INTERRUPCAO' = 'Interrupção Bombeio'
condicao_3 = (dfUni['Ocorrência'].str.contains('Bombeio')) & (dfUni['TIPO_INTERRUPCAO'] == 'Interrupção Bombeio')
dfUni.loc[condicao_3, 'INICIO_INTERRUPCAO'] = dfUni.loc[condicao_3, 'INICIO_INTERRUPCAO']
dfUni.loc[~condicao_3, 'TIPO_INTERRUPCAO'] = ''

# Manter valor da coluna 'FIM_INTERRUPCAO' onde 'Ocorrência' contém 'Bombeio' e 'TIPO_INTERRUPCAO' = 'Interrupção Bombeio'
condicao_4 = (dfUni['Ocorrência'].str.contains('Bombeio')) & (dfUni['TIPO_INTERRUPCAO'] == 'Interrupção Bombeio')
dfUni.loc[condicao_4, 'FIM_INTERRUPCAO'] = dfUni.loc[condicao_4, 'FIM_INTERRUPCAO']
dfUni.loc[~condicao_4, 'TIPO_INTERRUPCAO'] = ''

Duplicas as linhas em que acontece as interrupções e resetar os índices

In [250]:
df_filtered = dfUni[dfUni['TIPO_INTERRUPCAO'].notna()]
df_resultado = pandas.concat([dfUni, df_filtered], ignore_index=True)
df_resultado.reset_index(drop=True, inplace=True)
lista_remover = []
for i, row in df_resultado.iterrows():
    if i < len(df_resultado)-2:
        linha_atual = df_resultado.iloc[i]
        linha_seguinte = df_resultado.iloc[i+1]
        linha_seguinte_seguinte = df_resultado.iloc[i+2]
        if ((linha_atual['ID'] == linha_seguinte['ID'] and linha_atual['TIPO_INTERRUPCAO'] == "Interrupção Bombeio")) and ((linha_seguinte['ID'] == linha_seguinte_seguinte['ID'] and linha_seguinte['TIPO_INTERRUPCAO'] == "Interrupção Bombeio")):
            lista_remover.append(linha_seguinte_seguinte.name)


Ordenar por ID e Início

In [251]:
# Ordenar por ID e início
dfUni.sort_values(by=['ID', 'Início','INICIO_INTERRUPCAO'], inplace=True)

Se o inicio da interrupção for antes do bombeio, substituir o valor de inicio da interrupção pelo valor de inicio do bombeio, o mesmo acontece para as datas de fim, se o fim da interrupção for depois do fim do bombeio, substituir o fim da interrupção com o valor de fim do bombeio

In [252]:
import numpy as np

# Verificar se TIPO_INTERRUPCAO está vazio e substituir os valores das outras colunas
condicao = dfUni['TIPO_INTERRUPCAO'] == ''
dfUni.loc[condicao, ['DURACAO_INTERRUPCAO', 'INICIO_INTERRUPCAO', 'FIM_INTERRUPCAO']] = np.nan

# Substituir o início da interrupção pelo valor de início do bombeio onde necessário
dfUni.loc[dfUni['INICIO_INTERRUPCAO'] < dfUni['Início'], 'INICIO_INTERRUPCAO'] = dfUni['Início']

# Substituir o fim da interrupção pelo valor de fim do bombeio onde necessário
dfUni.loc[dfUni['FIM_INTERRUPCAO'] > dfUni['Término'], 'FIM_INTERRUPCAO'] = dfUni['Término']


Alterar as datas de término da primeira interrupção e início da segunda interrupção dentro de um mesmo id

In [253]:
# Verificar se TIPO_INTERRUPCAO de linhas consecutivas são iguais a "Interrupção Bombeio"
condicao_1 = (dfUni['TIPO_INTERRUPCAO'].shift() == 'Interrupção Bombeio') & (dfUni['TIPO_INTERRUPCAO'] == 'Interrupção Bombeio')
# Verificar se DURACAO_INTERRUPCAO de linhas consecutivas são diferentes
condicao_2 = (dfUni['TIPO_INTERRUPCAO'].shift() == 'Interrupção Bombeio') & (dfUni['TIPO_INTERRUPCAO'] == 'Interrupção Bombeio') & (dfUni['DURACAO_INTERRUPCAO'].shift() != dfUni['DURACAO_INTERRUPCAO'])
# Atualizar as datas conforme as condições especificadas
dfUni.loc[condicao_1, 'Término'] = dfUni['INICIO_INTERRUPCAO'].shift(-1)
dfUni.loc[condicao_1, 'Início'] = dfUni['FIM_INTERRUPCAO']
dfUni.loc[condicao_2, 'Início'] = dfUni['FIM_INTERRUPCAO']

In [254]:
dfUni.to_excel('c:\\Users\\caiobarreto\\Downloads\\teste.xlsx', index=False)
